## Neuron implementation

In [ ]:
from dataclasses import dataclass
from typing import Callable


@dataclass
class Neuron:
    w: float
    b: float
    f: Callable[[float], float]

    def activation(self, x):
        return self.f(self.w * x + self.b)

## One-dimensional array

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

@np.vectorize
def we_want_bigger_than_two(x):
    return 1 if x > 2 else 0


def sample_data(size):
    input = np.sort((np.random.rand(size) - 0.5) * 10)
    classification = we_want_bigger_than_two(input)
    return input, classification

input, classification = sample_data(40)
plt.scatter(input, np.zeros(input.size), c=classification)
plt.axvline(x = 2)

plt.show()

## Activation Function

In [ ]:
def f(x):
    return 1 if x > 0 else 0

# Learning

In [ ]:
def perceptron(input, classification):
    net = Neuron(0, 0, f)
    smart = False
    while not smart:
        smart = True
        for i in range(input.size):
            answer = net.activation(input[i])
            net.w = net.w + (classification[i] - answer) * input[i]
            net.b = net.b + (classification[i] - answer)
            smart = smart and answer == classification[i]
    return net


# Table output

In [ ]:
import pandas as pd


input, classification = sample_data(7)
net = perceptron(input, classification)
m = np.matrix((input, [net.activation(i) for i in input], classification))
print(pd.DataFrame(m))

# Visualized with graphs

On the left:
- green ticks show expected value
- blue ticks show current evaluation
- blue solid line show how network calculated values before activation function f is applied
- vertical dotted line show where data should be separated

On the right:
- plot of historical number of errors in the neuron

In [ ]:
from matplotlib.markers import CARETUP, CARETDOWN
from IPython import display

def perceptron(input, classification):
    fig, (ax_calc, ax_error) = plt.subplots(1, 2)
    fig.set_size_inches(16, 9)


    def plot(answer, error, net):
        @np.vectorize
        def value(i):
            return net.w * i + net.b

        ax_calc.cla()
        ax_calc.set_title(f'Network {net}')
        ax_error.set_title('Error')
        ax_calc.set_xlim([input[0], input[input.size - 1]])
        ax_calc.set_ylim([-3, 3])
        ax_calc.axvline(x = 2, dashes=(1, 1), linewidth=1)
        ax_calc.plot(input, value(input))
        ax_calc.scatter(input, classification, color='green', marker=CARETUP)
        ax_calc.scatter(input, answer, color='blue', marker=CARETDOWN)
        ax_error.plot(error)
        display.display(plt.gcf())
        display.clear_output(wait=True)


    net = Neuron(0, 0, f)
    smart = False
    errors = []
    answers = np.zeros(input.size)
    while not smart:
        smart = True
        for i in range(input.size):
            answers[i] = net.activation(input[i])
            net.w = net.w + (classification[i] - answers[i]) * input[i]
            net.b = net.b + (classification[i] - answers[i])
            errors.append(np.square(classification - answers).sum())
            smart = smart and answers[i] == classification[i]
            # Plot graphs after every single example because we don't expect learning to take very long time
            plot(answers, errors, net)

    return net

input, classification = sample_data(40)
net = perceptron(input, classification)